<a href="https://colab.research.google.com/github/zeynepsenatatli/MachineLearningExercises/blob/main/UE06_C6E7%268.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 7

Train and fine-tune a Decision Tree for the moons dataset by following these
steps:

a. Use make_moons(n_samples=10000, noise=0.4) to generate a moons dataset.

In [ ]:
from sklearn.datasets import make_moons

X_moons, y_moons = make_moons(n_samples=10000, noise=0.4, random_state=42)

b. Use train_test_split() to split the dataset into a training set and a test set.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_moons, y_moons,
                                                    test_size=0.2,
                                                    random_state=42)

c. Use grid search with cross-validation (with the help of the GridSearchCV class) to find good hyperparameter values for a DecisionTreeClassifier. Hint: try various values for max_leaf_nodes

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

params = {
    'max_leaf_nodes': list(range(2, 100)),
    'max_depth': list(range(1, 7)),
    'min_samples_split': [2, 3, 4]
}
grid_search_cv = GridSearchCV(DecisionTreeClassifier(random_state=42),
                              params,
                              cv=3)

In [ ]:
grid_search_cv.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=DecisionTreeClassifier(random_state=42),
             param_grid={'max_depth': [1, 2, 3, 4, 5, 6],
                         'max_leaf_nodes': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                            13, 14, 15, 16, 17, 18, 19, 20, 21,
                                            22, 23, 24, 25, 26, 27, 28, 29, 30,
                                            31, ...],
                         'min_samples_split': [2, 3, 4]})

In [ ]:
grid_search_cv.best_params_

{'max_depth': 6, 'max_leaf_nodes': 17, 'min_samples_split': 2}

d. Train it on the full training set using these hyperparameters, and measure your model’s performance on the test set.You should get roughly 85% to 87% accuracy.

In [ ]:
from sklearn.metrics import accuracy_score

y_pred = grid_search_cv.predict(X_test)
accuracy_score(y_test, y_pred)

0.8595

### Ekstra

Bu parametreler, `DecisionTreeClassifier` veya `DecisionTreeRegressor` gibi ağaç tabanlı bir modelin hiperparametre ayarlaması için kullanılabilir. Bu parametrelerin her biri, modelin ağaç yapısını kontrol eden farklı özellikleri ayarlamak için kullanılabilir.

- `max_leaf_nodes`: Ağacın en fazla yaprak düğüm sayısını belirler. Bu sayı arttıkça, ağaç daha karmaşık hale gelir.

- `max_depth`: Ağacın en fazla derinliğini belirler. Bu parametre arttıkça, ağaç daha karmaşık hale gelir ve eğitim setine daha iyi uyum sağlar. Ancak, aşırı öğrenme riski de artar.

- `min_samples_split`: Bir iç düğümün bölünmesi için gereken minimum örnek sayısını belirler. Bu parametre arttıkça, ağaç daha az karmaşık hale gelir ve daha genel bir model olur. Ancak, bu parametrenin aşırı artması, ağacın eğitim verilerine kötü uyum sağlamasına neden olabilir.

Yukarıdaki parametreler, belirli bir modelin performansını etkileyebilir ve iyi bir sonuç elde etmek için optimal değerleri seçmek önemlidir. `GridSearchCV` yöntemi gibi hiperparametre arama yöntemleri, bu parametrelerin optimal değerlerini belirlemek için kullanılabilir.

### 8

Grow a forest by following these steps.

a. Continuing the previous exercise,generate 1,000 subsets of the training set, each containing 100 instances selected randomly. Hint: you can use Scikit-Learn’s ShuffleSplit class for this.





In [ ]:
from sklearn.model_selection import ShuffleSplit

n_trees = 1000
n_instances = 100

mini_train_sets = []

rs = ShuffleSplit(n_splits=n_trees, test_size=len(X_train) - n_instances, random_state=42)

for mini_train_index, mini_test_index in rs.split(X_train):
    X_mini_train = X_train[mini_train_index]
    y_mini_train = y_train[mini_train_index]
    mini_train_sets.append((X_mini_train, y_mini_train))

In [ ]:
len(X_mini_train)

100

In [ ]:
len(mini_test_index)

7900

b. Train one Decision Tree on each subset, using the best hyperparameter values found in the previous exercise. Evaluate these 1,000 Decision Trees on the test set. Since they were trained on smaller sets, these Decision Trees will likely perform worse than the first Decision Tree, achieving only about 80% accuracy.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

forest = []
for i in range(n_trees):
    tree = DecisionTreeClassifier(max_depth= 6, max_leaf_nodes= 17, min_samples_split= 2)
    forest.append(tree)

accuracy = []

for tree, (X_mini_train, y_mini_train) in zip(forest, mini_train_sets):
    tree.fit(X_mini_train, y_mini_train)

    y_pred = tree.predict(X_test)
    accuracy.append(accuracy_score(y_test, y_pred))

In [ ]:
import numpy as np

np.mean(accuracy)

0.805253

c. Now comes the magic. For each test set instance, generate the predictions of the 1,000 Decision Trees, and keep only the most frequent prediction (you can use SciPy’s mode() function for this). This approach gives you majority-vote predictions over the test set.

In [ ]:
Y_pred = []
index = 0
for tree in forest:
  Y_pred.append(tree.predict(X_test))

In [ ]:
Y_pred = np.array(Y_pred)

In [ ]:
Y_pred

array([[0, 1, 0, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 0, ..., 1, 0, 0],
       ...,
       [1, 1, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 1],
       [1, 1, 0, ..., 0, 0, 0]])

In [ ]:
Y_pred.shape

(1000, 2000)

In [ ]:
from scipy.stats import mode

y_pred_majority_votes, n_votes = mode(Y_pred, axis=0)

<ipython-input-46-725f07c27fd0>:3: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  y_pred_majority_votes, n_votes = mode(Y_pred, axis=0)


In [ ]:
y_pred_majority_votes

array([[1, 1, 0, ..., 0, 0, 0]])

In [ ]:
n_votes

array([[950, 902, 958, ..., 921, 992, 618]])

d. Evaluate these predictions on the test set: you should obtain a slightly higher accuracy than your first model (about 0.5 to 1.5% higher). Congratulations, you have trained a Random Forest classifier.

In [ ]:
accuracy_score(y_test, y_pred_majority_votes.reshape([-1]))

0.872